In [ ]:
"""
Burada kendi Google Drive'ımızı bağlıyoruz.
"""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Gerekli kütüphanelerin import edilmesi

In [ ]:
"""
Gerekli kütüphaneleri ekliyoruz.
"""

import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import os 

from sklearn.preprocessing import LabelEncoder, minmax_scale
from sklearn.model_selection import train_test_split

## Dosyalama formatının oluşturulması
Gerekli ön işlemelerin yapıldığı resimlerin saklanacağı dosyaların oluşturulması.

In [ ]:
"""
Burada 7. satırda yer alan path kendi driveımızdaki herhangi bir yer olabilir.
Kod öncelikle belirlediğimiz yerin (Bu bende Global_AI_Hub adında bir klasör) içerisinde
'dataset' adında bir klasör oluşturacak ve onun içinede 0 dan 9 a kadar klasörler oluşturacak.
"""

os.mkdir(f"/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/dataset")
for i in range(10):
  os.mkdir(f"/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/dataset/{i}")

## GrayScale dönüşümü, resize ve normalizasyon

In [ ]:
"""
Burada her resmin yolunu ve etiketini eklemek için data adında bir dictionary oluşturup
onun içerisinede iki tane liste oluşturuyoruz. Image resmin yolu, label ise etiket.
"""

data = dict()
data['image'] = []
data['label'] = []

include = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [ ]:
"""
Burada Global AI Hub tarafından hazır halde paylaşılan 'spectograms' klasörünün kendi driveınızda bir kısayolunun bulunması gerek.
Yaptığımız şey ise o klasörde yer alan her bir resmin yolunu ve etiketini alıp oluşturduğumuz dictionarye eklemek.
"""

for subdir in os.listdir('/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/spectrograms/'):
        if subdir in include:
            current_path = os.path.join('/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/spectrograms/', subdir)
            for file in os.listdir(current_path):
              data['image'].append(os.path.join(current_path, file))
              data['label'].append(subdir)

In [ ]:
"""
Pandas ile dataframe dönüşümü yapıyoruz bu bize hem görselleştirme anlamında hemde kullanış açısından kolaylık sağlıyor.
"""

df = pd.DataFrame(data)
df

,image,label
0,/content/drive/MyDrive/Colab_Notebooks/Global_...,6
1,/content/drive/MyDrive/Colab_Notebooks/Global_...,6
2,/content/drive/MyDrive/Colab_Notebooks/Global_...,6
3,/content/drive/MyDrive/Colab_Notebooks/Global_...,6
4,/content/drive/MyDrive/Colab_Notebooks/Global_...,6
...,...,...
8727,/content/drive/MyDrive/Colab_Notebooks/Global_...,7
8728,/content/drive/MyDrive/Colab_Notebooks/Global_...,7
8729,/content/drive/MyDrive/Colab_Notebooks/Global_...,7
8730,/content/drive/MyDrive/Colab_Notebooks/Global_...,7


In [ ]:
"""
Dataframe sayesinde datanın dağılımını görüyoruz
"""

print('The distribution of the classes:')
print(df['label'].value_counts())

The distribution of the classes:
5    1000
2    1000
3    1000
9    1000
0    1000
4    1000
7    1000
8     929
1     429
6     374
Name: label, dtype: int64


In [ ]:
"""
Burada oluşturduğumuz dataframe üzerinden Globa AI Hub tarafından paylaşılan orijinal 'spectograms' klasöründe bulunan her bir resmi önce okuyoruz, grayscale dönüşümü yapıyoruz 
ardından resize ettikten sonra son olarak kendi driveımızda oluşturduğumuz 'dataset' klasöründe yer alan uygun klasörlere kaydediyoruz.
Yani kısaca paylaşılan PDFde istenen bütün işlemlerin uygulandığı resimleri kendi driveımıza alıyoruz ki training notebookunda kullanabilelim. 

Bu for döngüsü biraz uzun sürebilir çünkü 8732 adet resim bulunmakta
"""

for index, row in df.iterrows():
  image = cv.imread(f"{df['image'][index]}")
  gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  resize = cv.resize(gray, (256, 256))
  resize = resize/255
  cv.imwrite(f"/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/dataset/{df['label'][index]}/{os.path.basename(df['image'][index])}", resize)

# Ön işlem yapılan resimlerin kontrol edilmesi için dataframe oluşturma

In [ ]:
"""
Elde ettiğimiz yeni resimlerin düzgün bir şekilde işlendiğinden ve dağıtıldığından emin olmak için 
yukarıda oluşturduğumuz dataframe gibi bir dataframe daha oluşturup iki dataframe'i karşılaştırarak
kontrol ediyoruz
"""

data_normalized = dict()
data_normalized['image'] = []
data_normalized['label'] = []

include = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [ ]:
for subdir in os.listdir('/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/dataset/'):
        if subdir in include:
            current_path = os.path.join('/content/drive/MyDrive/Colab_Notebooks/Global_AI_Hub/dataset/', subdir)
            for file in os.listdir(current_path):
              data_normalized['image'].append(os.path.join(current_path, file))
              data_normalized['label'].append(subdir)

In [ ]:
df_normalized = pd.DataFrame(data_normalized)
df_normalized

,image,label
0,/content/drive/MyDrive/Colab_Notebooks/Global_...,0
1,/content/drive/MyDrive/Colab_Notebooks/Global_...,0
2,/content/drive/MyDrive/Colab_Notebooks/Global_...,0
3,/content/drive/MyDrive/Colab_Notebooks/Global_...,0
4,/content/drive/MyDrive/Colab_Notebooks/Global_...,0
...,...,...
8727,/content/drive/MyDrive/Colab_Notebooks/Global_...,9
8728,/content/drive/MyDrive/Colab_Notebooks/Global_...,9
8729,/content/drive/MyDrive/Colab_Notebooks/Global_...,9
8730,/content/drive/MyDrive/Colab_Notebooks/Global_...,9


In [ ]:
print('The distribution of the classes:')
print(df_normalized['label'].value_counts())

The distribution of the classes:
0    1000
2    1000
3    1000
4    1000
5    1000
7    1000
9    1000
8     929
1     429
6     374
Name: label, dtype: int64


In [ ]:
"""
Bu hücre ve bundan sonraki diğer iki hücre sadece classlar arasındaki dağılımın doğru bir şekilde
yapılıp yapılmadığını kontrol etmek için oluşturulmuştur. Training notebookunda random_state=42 kullanarak
train test split'i tekrar yapıyor olacağız.
"""

x_main, x_test, y_main, y_test = train_test_split(df_normalized.image, df_normalized.label, test_size = 0.1, random_state = 42, stratify = df_normalized.label)
x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size = 0.1, random_state = 42, stratify = y_main)

In [ ]:
df_train = pd.concat([x_train, y_train], axis=1)
df_val = pd.concat([x_val, y_val], axis=1)
df_test = pd.concat([x_test, y_test], axis=1)

In [ ]:
print('The distribution of the classes:')
print('---------------------------------------------------')
print('TRAIN SET: ')
print(df_train['label'].value_counts())
print('---------------------------------------------------')
print('TEST SET: ')
print(df_test['label'].value_counts())
print('---------------------------------------------------')
print('VALIDATION SET: ')
print(df_val['label'].value_counts())

The distribution of the classes:
---------------------------------------------------
TRAIN SET: 
3    810
9    810
7    810
4    810
2    810
5    810
0    810
8    752
1    347
6    303
Name: label, dtype: int64
---------------------------------------------------
TEST SET: 
5    100
2    100
9    100
7    100
3    100
0    100
4    100
8     93
1     43
6     38
Name: label, dtype: int64
---------------------------------------------------
VALIDATION SET: 
2    90
7    90
4    90
3    90
0    90
5    90
9    90
8    84
1    39
6    33
Name: label, dtype: int64
